In [1]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
import timeit
import missingno as msno
from skimage import morphology
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from PIL import Image
from collections import Counter

In [2]:
image_folder = '../../data/raw/example_image/'
segmentation_folder = '../../data/raw/example_segmentation/'

extra_image_folder = '../../data/raw/extra_keratosis'
extra_segmentation_folder = '../../data/raw/extra_segmentation'


extra_ground_truth = '../../data/raw/ISIC-2017_Training_Part3_GroundTruth.csv'
ground_truth = '../../data/raw/example_ground_truth.csv'
features = '../../data/features/features.csv'

In [3]:
image_files = os.listdir(image_folder)
segmentation_files = os.listdir(segmentation_folder)

extra_image_files = os.listdir(extra_image_folder)
extra_segmentation_files = os.listdir(extra_segmentation_folder)

extra_gt = pd.read_csv(extra_ground_truth)
ground_truth = pd.read_csv(ground_truth)
features = pd.read_csv(features)

In [4]:
ground_truth = ground_truth.set_index('image_id')
features = features.set_index('id')
extra_gt = extra_gt.set_index('image_id')

In [5]:
color_images_paths = []
segmentation = []

# getting the image_paths
for i in image_files[1:]:
    image_path= os.path.join(image_folder,i)
    if not i.endswith('superpixels.png'):
        color_images_paths.append(image_path)
for i in segmentation_files:
    image_path1 = os.path.join(segmentation_folder,i)
    segmentation.append(image_path1)

# would be nice to create a function that actually checks if the tuple contains the right colored image and segemntation one
both_images = list(zip(color_images_paths,segmentation))

#splitting the images
keratosis_images = ground_truth.index[ground_truth['seborrheic_keratosis'] == 1.0].tolist()
melanoma_images = ground_truth.index[ground_truth['melanoma'] == 1.0].tolist()
healthy_images = ground_truth[(ground_truth['seborrheic_keratosis'] == 0.0) & (ground_truth['melanoma'] == 0.0)]
#print(healthy_images.index.tolist) - to get only the healthy images in a list hence the healthy_images is a df

keratosis = [os.path.join(image_folder,i + '.jpg') for i in keratosis_images] #positive keratosis colour_images path
melanoma = [os.path.join(image_folder,i + '.jpg') for i in melanoma_images] #positive melanoma colour_images path
healthy = [os.path.join(image_folder,i + '.jpg') for i in healthy_images.index.tolist()] #healthy colour_images path

k_segmentation = [os.path.join(segmentation_folder,i + '_segmentation.png') for i in keratosis_images] #positive keratosis segmentation path
m_segmentation = [os.path.join(segmentation_folder,i + '_segmentation.png') for i in melanoma_images] #positive melanoma segmentation path
h_segmentation = [os.path.join(segmentation_folder,i + '_segmentation.png') for i in healthy_images.index.tolist()]#healthy segmentation path"

In [6]:
final_keratosis = list(zip(keratosis,k_segmentation))
final_melanoma = list(zip(melanoma,m_segmentation))
final_healthy = list(zip(healthy,h_segmentation))

extra_keratosis = [os.path.join(extra_image_folder,i) for i in extra_image_files]
extra_segmentation = [os.path.join(extra_segmentation_folder,i) for i in extra_segmentation_files ]
final_extra = list(zip(extra_keratosis,extra_segmentation))

In [7]:
Keratosis_all = final_keratosis + final_extra

In [8]:
segm_k_all = k_segmentation + extra_segmentation

Some fucking bullshit haha
The images has valid values, but there is some big difference between some of them.
Example would be the image "ISIC_0014616", where the lines and edges in the image is very undetectable, it has very low value then which means it is a blurry image. Though that when applying a blur filter to control if the edge detection works, it gives an opposite value and increases the edge detection...

In [17]:
def edge_detect(img):
    
    
    test_image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)

    image_unblur = cv2.resize(test_image, (500, 400))
    
    imageB = cv2.GaussianBlur(image_unblur, (5,5), 0) #Bias for if images are considered blurry

    #Laplacian.var() works in such a way that it checks for defined edges, how many clear edges/lines is present
    img_var = cv2.Laplacian(image_unblur, cv2.CV_64F, ksize=11).var()
    img_var_blur = cv2.Laplacian(imageB, cv2.CV_64F, ksize=11).var()

    if img_var < 100000000000:
        print(img_var, "variance value is less than 100 milliard, the image is considered blurry")
    else:
        print(img_var, "variance value is above 100 milliard, image is not considered blurry")

    #Prints the variance value of the blur filter we make in the code, not needed
    #print(img_var_blur, "variance value of an image with gaussian blur filter on")

    
    cv2.imshow("Original image no blur", image_unblur)
    cv2.imshow("Blur filter", imageB)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return

edge_detect("../../Data/Raw/example_image/ISIC_0006671.jpg")

98953437900.10329 variance value is less than 100 milliard, the image is considered blurry
